In [27]:
import numpy as np
from torchvision.datasets import MNIST
from sklearn.preprocessing import OneHotEncoder
import math
from scipy.stats import bernoulli
from sklearn.metrics import accuracy_score

In [28]:
def init_dataset(is_train: bool):
    dataset = MNIST(
        root='./data',
        train=is_train,
        transform=lambda x: np.array(x).flatten(),
        download=False
    )

    mnist_data = []
    mnist_labels = []
    for image,label in dataset:
        mnist_data.append(image)
        mnist_labels.append(label)

    return np.array(mnist_data), np.array(mnist_labels)

In [29]:
def one_hot_encode(labels):
    encoder = OneHotEncoder(sparse_output=False)
    return encoder.fit_transform(labels.reshape(-1, 1))

In [30]:
def weight_initialization_Xavier_Uniform(input_dim, output_dim):
    low_bound = -math.sqrt(6/(input_dim + output_dim))
    upper_bound = math.sqrt(6/(input_dim + output_dim))
    W = np.random.uniform(low_bound, upper_bound, size=(input_dim, output_dim))
    b = np.zeros((1, output_dim))
    return W, b

In [31]:
def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return expZ / np.sum(expZ, axis=1, keepdims=True)

def sigmoid(Z):
    return 1/(1 + np.exp(-Z))

In [32]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    
    return W1, b1, W2, b2

In [33]:
def compute_loss_cross_entropy(A, Y):
    m = Y.shape[0]
    log_likelihood = -np.log(A[range(m), Y.argmax(axis=1)])
    return np.sum(log_likelihood) / m

In [34]:
def forward_propagation_no_dropout(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)

    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    return A1, A2

In [35]:
def predict(X, W1, b1, W2, b2):
    A2 = forward_propagation_no_dropout(X, W1, b1, W2, b2)[1]
    return np.argmax(A2, axis=1)

In [ ]:
train_X, train_Y = init_dataset(True)
test_X, test_Y = init_dataset(False)
    
train_Y = one_hot_encode(train_Y)
test_Y_one_hot = one_hot_encode(test_Y)

epochs = 100
learning_rate = 0.01
batch_size = 100
dropout_rate = 0.20
    
W1, b1 = weight_initialization_Xavier_Uniform(784, 100)
W2, b2 = weight_initialization_Xavier_Uniform(100, 10)

for epoch in range(epochs):
    perm = np.random.permutation(train_X.shape[0])
    train_X = train_X[perm]
    train_Y = train_Y[perm]
    
    for i in range(0, train_X.shape[0], batch_size):
        X_batch = train_X[i:i+batch_size]
        Y_batch = train_Y[i:i+batch_size]
        
        # Forward propagation
        Z1 = np.dot(X_batch, W1) + b1
        Y1 = sigmoid(Z1)
        dropouts = bernoulli.rvs(dropout_rate, size=Y1.shape) 
        Y1_dropout = Y1 * dropouts / (1 - dropout_rate)
        Z2 = np.dot(Y1_dropout, W2) + b2
        Y2 = softmax(Z2)
        
        # Compute loss (optional for tracking)
        loss = compute_loss_cross_entropy(Y2, Y_batch)
        
        # Backward propagation
        m = X_batch.shape[0]

        error_Z2 = Y2 - Y_batch
        dW2 = np.dot(Y1.T, error_Z2) / m  
        db2 = np.sum(error_Z2, axis=0, keepdims=True) / m

        error_Z1 = np.dot(error_Z2, W2.T) * Y1 * (1 - Y1)
        
        error_Z1 = error_Z1 * dropouts
        error_Z1 /= (1 - dropout_rate)
    
        dW1 = np.dot(X_batch.T, error_Z1) / m 
        db1 = np.sum(error_Z1, axis=0, keepdims=True) / m  
        
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
        
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}')


train_predictions = predict(train_X, W1, b1, W2, b2)
test_predictions = predict(test_X, W1, b1, W2, b2)
    
print(f'Training Accuracy: {accuracy_score(np.argmax(train_Y, axis=1), train_predictions) * 100:.2f}%')
print(f'Test Accuracy: {accuracy_score(test_Y, test_predictions) * 100:.2f}%')